In [1]:
from src.annotation.doccano import get_latest_annotated_data
from src.data.preprocessing import DataPreprocessorHelland
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import matplotlib.pyplot as plt
import tensorflow as tf
from datasets import Dataset, DatasetDict
import ray.data
from ray.data.preprocessors import BatchMapper
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import evaluate
from src.slack_alert.sofus_alert import sofus_alert
from huggingface_hub import login
import configparser
from src.utils import get_project_root

2023-03-20 11:10:02.570977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 11:10:11.890460: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 11:10:11.890478: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 11:10:28.837450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data = get_latest_annotated_data().loc[:, ['text', 'label']].dropna().reset_index()
data = data.loc[:, ['text', 'label']]
data_preprocessor = DataPreprocessorHelland(data['text'])
data_preprocessor.make_lower_cased()
data_preprocessor.remove_extra_spaces()
data['text'] = data_preprocessor.text_series
one_hot = MultiLabelBinarizer()
one_hot_label = one_hot.fit_transform(data['label'])
one_hot_label = [list(map(float, x)) for x in one_hot_label]
data['label'] = pd.Series(list(one_hot_label))
data

,text,label
0,premier league valgte å offentliggjøre tre fly...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
1,romelu lukaku har uttalt at han er misfornøyd ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"– la oss være ærlige. jeg liker det ikke, for ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,manchester united har fått mye kritikk etter å...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,her er torsdagens oddstips!,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, ..."
...,...,...
2004,marko gruljic fra red star belgrade 6. januar ...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2005,tv 2s fotballkommentator øyvind alsaker mener ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2006,de fire beste i premier league får spille i ch...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2007,"– virgil van dijk, alisson becker og mohamed s...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [3]:
train_split, validation_split = train_test_split(data, test_size=0.2)

In [4]:
train_dataset = Dataset.from_pandas(train_split)
train_dataset = train_dataset.remove_columns(['__index_level_0__'])
validation_dataset = Dataset.from_pandas(validation_split)
validation_dataset = validation_dataset.remove_columns(['__index_level_0__'])
split_dict = {'train': train_dataset, 'validation': validation_dataset}
datasets = DatasetDict(split_dict)
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1607
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 402
    })
})

In [5]:
ray_datasets = ray.data.from_huggingface(datasets)
ray_datasets

2023-03-20 11:11:21,721	INFO worker.py:1553 -- Started a local Ray instance.


{'train': Dataset(num_blocks=1, num_rows=1607, schema={text: string, label: list<item: double>}),
 'validation': Dataset(num_blocks=1, num_rows=402, schema={text: string, label: list<item: double>})}

In [6]:
model_checkpoint = 'NbAiLab/nb-bert-large'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, model_max_length=512)

In [1]:
def preprocess_function(examples):

    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

encoded_dataset = datasets.map(preprocess_function, batched=True)

NameError: name 'datasets' is not defined

In [8]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]
task = 'multi_label_classification'
batch_size = 16

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [9]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [10]:
model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=22, problem_type="multi_label_classification")

Some weights of the model checkpoint at NbAiLab/nb-bert-large were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

In [11]:
root = get_project_root()
config = configparser.ConfigParser()
config.read(root + '/src/modelling/my_config.ini')
token = config['huggingface']['TOKEN']

login(token=token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /home/eirik/.cache/huggingface/token
Login successful


In [12]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

(raylet) [2023-03-20 11:12:21,682 E 3908 3908] (raylet) node_manager.cc:3040: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 291938993b41439b355353cf65a598aa3c137dc9c71853926691c687, IP: 192.168.1.28) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.1.28`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
Cloning https://huggingface.co/hellund/nb-bert-large-finetuned-multi_label_classification into local empty directory.


Download file pytorch_model.bin:   0%|          | 9.14k/1.32G [00:00<?, ?B/s]

Download file runs/Feb28_14-47-02_LAPTOP-2G8VCJEJ/events.out.tfevents.1677592043.LAPTOP-2G8VCJEJ.32188.0: 100%…

Download file runs/Feb28_14-47-02_LAPTOP-2G8VCJEJ/1677601062.8188286/events.out.tfevents.1677601062.LAPTOP-2G8…

Download file runs/Feb28_12-37-52_LAPTOP-2G8VCJEJ/1677584423.8420374/events.out.tfevents.1677584423.LAPTOP-2G8…

...

Clean file runs/Feb28_13-52-09_LAPTOP-2G8VCJEJ/events.out.tfevents.1677588739.LAPTOP-2G8VCJEJ.34556.0:  21%|##…

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Clean file runs/Feb28_13-10-33_LAPTOP-2G8VCJEJ/events.out.tfevents.1677586245.LAPTOP-2G8VCJEJ.35264.0:  21%|##…

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.32G [00:00<?, ?B/s]

In [13]:
trainer.train()
sofus_alert()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/eirik/miniconda3/envs/pytorch/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1607
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 505
  Number of trainable parameters = 355109910
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is 

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 2.94 GiB total capacity; 2.28 GiB already allocated; 3.88 MiB free; 2.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 401
  Batch size = 16


{'eval_loss': 0.13401861488819122,
 'eval_f1': 0.647780925401322,
 'eval_roc_auc': 0.7626107290024299,
 'eval_accuracy': 0.428927680798005,
 'eval_runtime': 215.6616,
 'eval_samples_per_second': 1.859,
 'eval_steps_per_second': 0.121,
 'epoch': 5.0}

In [54]:
text = '- Jeg elsker å score mål på corner. Vi var mye bedre i luften enn Brann idag'
encoding = tokenizer(text, return_tensors="pt")

outputs = trainer.model(**encoding)

In [61]:
logits = outputs.logits

In [62]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predictions = np.reshape(predictions, (1, predictions.shape[0]))
predicted_labels = one_hot.inverse_transform(predictions)
print(predicted_labels)

[('Quote',)]
